In [15]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName(
	'Read CSV File into DataFrame').getOrCreate()

df = spark.read.csv('C:/Users/jorda/Documents/GitHub/SDSC-BigData/daily_weather/daily_weather.csv', sep=',',
						inferSchema=True, header=True)

In [16]:
df.show(5)

+------+-----------------+------------------+----------------------+------------------+----------------------+------------------+---------------------+-----------------+---------------------+---------------------+
|number| air_pressure_9am|      air_temp_9am|avg_wind_direction_9am|avg_wind_speed_9am|max_wind_direction_9am|max_wind_speed_9am|rain_accumulation_9am|rain_duration_9am|relative_humidity_9am|relative_humidity_3pm|
+------+-----------------+------------------+----------------------+------------------+----------------------+------------------+---------------------+-----------------+---------------------+---------------------+
|     0|918.0600000000087| 74.82200000000041|                 271.1| 2.080354199999768|    295.39999999999986| 2.863283199999908|                  0.0|              0.0|    42.42000000000046|   36.160000000000494|
|     1|917.3476881177097| 71.40384263106537|    101.93517935618371|2.4430092157340217|    140.47154847112498|3.5333236016106238|               

In [17]:
df.printSchema()

root
 |-- number: integer (nullable = true)
 |-- air_pressure_9am: double (nullable = true)
 |-- air_temp_9am: double (nullable = true)
 |-- avg_wind_direction_9am: double (nullable = true)
 |-- avg_wind_speed_9am: double (nullable = true)
 |-- max_wind_direction_9am: double (nullable = true)
 |-- max_wind_speed_9am: double (nullable = true)
 |-- rain_accumulation_9am: double (nullable = true)
 |-- rain_duration_9am: double (nullable = true)
 |-- relative_humidity_9am: double (nullable = true)
 |-- relative_humidity_3pm: double (nullable = true)



In [18]:
df.describe('avg_wind_speed_9am').show()

+-------+------------------+
|summary|avg_wind_speed_9am|
+-------+------------------+
|  count|              1092|
|   mean|  5.50828424225493|
| stddev|4.5528134655317185|
|    min|  0.69345139999974|
|    max|23.554978199999763|
+-------+------------------+



In [20]:
df.count()

1095

In [19]:
df.na.drop(subset=['air_pressure_9am']).count()

1092

In [21]:
df.stat.corr('relative_humidity_9am', 'relative_humidity_3pm')

0.8830741909793918

In [22]:
df.pandas_api(index_col='number').head()

,air_pressure_9am,air_temp_9am,avg_wind_direction_9am,avg_wind_speed_9am,max_wind_direction_9am,max_wind_speed_9am,rain_accumulation_9am,rain_duration_9am,relative_humidity_9am,relative_humidity_3pm
number,,,,,,,,,,
0,918.060000,74.822000,271.100000,2.080354,295.400000,2.863283,0.0,0.0,42.420000,36.160000
1,917.347688,71.403843,101.935179,2.443009,140.471548,3.533324,0.0,0.0,24.328697,19.426597
2,923.040000,60.638000,51.000000,17.067852,63.700000,22.100967,0.0,20.0,8.900000,14.460000
3,920.502751,70.138895,198.832133,4.337363,211.203341,5.190045,0.0,0.0,12.189102,12.742547
4,921.160000,44.294000,277.800000,1.856660,136.500000,2.863283,8.9,14730.0,92.410000,76.740000


In [23]:
df.describe().pandas_api()

,summary,number,air_pressure_9am,air_temp_9am,avg_wind_direction_9am,avg_wind_speed_9am,max_wind_direction_9am,max_wind_speed_9am,rain_accumulation_9am,rain_duration_9am,relative_humidity_9am,relative_humidity_3pm
0,count,1095,1092,1090,1091,1092,1092,1091,1089,1092,1095,1095
1,mean,547.0,918.8825513138094,64.93300141287072,142.2355107005759,5.50828424225493,148.95351796516923,7.019513529175272,0.20307895225211126,294.1080522756142,34.24140205923536,35.34472714825898
2,stddev,316.24357700987383,3.184161180386833,11.175514003175877,69.13785928889189,4.5528134655317185,67.23801294602953,5.598209170780958,1.5939521253574893,1598.0787786601481,25.472066802250055,22.524079453587273
3,min,0,907.9900000000024,36.752000000000685,15.500000000000046,0.69345139999974,28.89999999999991,1.1855782000000479,0.0,0.0,6.090000000001012,5.3000000000006855
4,max,1094,929.3200000000012,98.90599999999992,343.4,23.554978199999763,312.19999999999993,29.84077959999996,24.01999999999907,17704.0,92.6200000000002,92.2500000000003


In [24]:
df.describe('air_temp_9am').show()

+-------+------------------+
|summary|      air_temp_9am|
+-------+------------------+
|  count|              1090|
|   mean| 64.93300141287072|
| stddev|11.175514003175877|
|    min|36.752000000000685|
|    max| 98.90599999999992|
+-------+------------------+



In [25]:
removeAllDF = df.na.drop()

In [26]:
removeAllDF.describe('air_temp_9am').show()

+-------+------------------+
|summary|      air_temp_9am|
+-------+------------------+
|  count|              1064|
|   mean| 65.02260949558733|
| stddev|11.168033449415704|
|    min|36.752000000000685|
|    max| 98.90599999999992|
+-------+------------------+



In [27]:
removeAllDF.count()

1064

In [28]:
from pyspark.sql.functions import avg

imputeDF = df

In [29]:
for x in imputeDF.columns:
  meanValue = removeAllDF.agg(avg(x)).first()[0]
  print(x, meanValue)
  imputeDF = imputeDF.na.fill(meanValue, [x])

number 545.0018796992481
air_pressure_9am 918.9031798641051
air_temp_9am 65.02260949558733
avg_wind_direction_9am 142.30675564934037
avg_wind_speed_9am 5.48579305071369
max_wind_direction_9am 148.48042413321315
max_wind_speed_9am 6.999713658875691
rain_accumulation_9am 0.18202347650615522
rain_duration_9am 266.3936973996037
relative_humidity_9am 34.07743985327709
relative_humidity_3pm 35.14838093290533


In [30]:
imputeDF.describe('air_temp_9am').show()

+-------+------------------+
|summary|      air_temp_9am|
+-------+------------------+
|  count|              1095|
|   mean| 64.93341058219818|
| stddev| 11.14994819992023|
|    min|36.752000000000685|
|    max| 98.90599999999992|
+-------+------------------+

